In [81]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
from matplotlib import gridspec

import numpy as np
import scipy.sparse as sp
import numpy.linalg as la

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
def fovX_from_fovY(fovY, aspect):
    tanY = np.tan(fovY / 2.0)
    tanX = tanY * aspect;
    fovX = np.arctan(tanX)
    return fovX

def drawSetup(fudgeX=10, fudgeY_pos=6,fudgeY_neg=8):
    pylab.rcParams['figure.figsize'] = 8, 8

    plt.axes().set_aspect('equal')
    fig = plt.gcf()
    
    # Set up plot size
    plt.axes().set_xlim((np.min([PA.x(), PB.x()])-fudgeX,np.max([PA.x(), PB.x()])+fudgeX))
    plt.axes().set_ylim((np.min([PA.y(), PB.y()])-fudgeY_neg,np.max([PA.y(), PB.y()])+fudgeY_pos))

    # Draw People Positions
    plt.scatter([PA.x(), PB.x()],[PA.y(), PB.y()],c="red",linewidths=0)
    line_AB = plt.Line2D([PA.x(), PB.x()],[PA.y(), PB.y()], c="black",alpha=0.3)
    fig.gca().add_artist(line_AB)

    # Draw Circles    
    circle_PA=plt.Circle((PA.x(),PA.y()),min_dist,color='g',alpha=0.5)
    circle_PB=plt.Circle((PB.x(),PB.y()),min_dist,color='g',alpha=0.5)
    
    fig.gca().add_artist(circle_PA)
    fig.gca().add_artist(circle_PB)

    plt.annotate(s="A", xy=(PA.x(),PA.y()),xytext=(3,4),textcoords="offset points")
    plt.annotate(s="B", xy=(PB.x(),PB.y()),xytext=(3,4),textcoords="offset points")
    
    # Draw Camera positions
    plt.scatter([C.x()],[C.y()],c="blue",linewidths=0)
    plt.annotate(s="C", xy=(C.x(), C.y()),xytext=(3,4),textcoords="offset points")

In [83]:
# Conversion from world space to camera screen space (from Unity: z,x,-y)

# simple test
# camera specs (position, forward, up, fov)
C_test = np.array([1,0,0])#-92.9,-39.6,-2.1])
forward_test = np.array([-1,0,0])#-1.0,0.0,0.0])
up_test = np.array([0,1,0])#0.0,0.0,-1.0])
aspect_test = 16.0/9.0
fovY_test = 0.8726646
fovX_test = fovX_from_fovY(fovY_test, aspect_test)

# scene specs
PA_test = np.array([-1,0,0])#-98.2,-40.6,-1.6])     # person A position

In [87]:
# https://www.opengl.org/sdk/docs/man2/xhtml/gluPerspective.xml
# https://unspecified.wordpress.com/2012/06/21/calculating-the-gluperspective-matrix-and-other-opengl-matrix-maths/
def perspective_matrix(fovY, aspect, near, far):
    print fovY
    print aspect
    f = 1.0 / np.tan (fovY / 2.0)
    p_matrix = np.matrix([[f/aspect, 0.0, 0.0, 0.0],
                          [0.0, f, 0.0, 0.0],
                          [0.0, 0.0, (far+near)/(near-far), (2.0*far*near)/(near-far)], 
                          [0.0, 0.0, -1.0, 0.0]])
    return p_matrix

proj_test = perspective_matrix(fovY_test, aspect_test, 0.1, 100.0)
print "proj"
print proj_test

0.8726646
1.77777777778
proj
[[ 1.20628518  0.          0.          0.        ]
 [ 0.          2.14450699  0.          0.        ]
 [ 0.          0.         -1.002002   -0.2002002 ]
 [ 0.          0.         -1.          0.        ]]


In [85]:
# VECTOR<float,2> OPENGL_WORLD::
# Project_World_Space_Point_Onto_Image_Plane(const TV& world_space_point)
# {
#     MATRIX<float,4> perspective_matrix;
#     perspective_matrix(1,1)=(1/tan(.5f*fovy*(float)pi/180))*(float)window-      >Height()/(float)window->Width();
#     perspective_matrix(2,2)=1/tan(.5f*fovy*(float)pi/180);
#     perspective_matrix(3,3)=(farclip+nearclip)/(nearclip-farclip);
#     perspective_matrix(3,4)=(2*farclip*nearclip)/(nearclip-farclip);
#     perspective_matrix(4,3)=-1;

#     VECTOR<T,3> eye_space_point=TV(0.f,0.f,-camera_distance)+(arcball_matrix).Homogeneous_Times(world_space_point-target_position);
#     VECTOR<T,3> image_space_point=perspective_matrix.Homogeneous_Times(eye_space_point);
#     image_space_point(1)=((float)window->Width())*image_space_point(1)/2+((float)window->Width())/2;
#     image_space_point(2)=((float)window->Height())*image_space_point(2)/2+((float)window->Height())/2;
#     return VECTOR<float,2>(image_space_point(1),image_space_point(2));
# }

look_test = C_test + forward_test # target position
print "look = ", look_test
print "C = ", C_test
print "forward = ", forward_test
print "up = ", up_test
right_test = np.cross(up_test, forward_test)
right_test / la.norm(right_test)
print "right = ", right_test

# axes to camera space
def arcball_matrix(camForward, camUp):
    z = -camForward
    z = z / la.norm(z) # forward
    y = camUp # up
    x = np.cross(y, z) # right
    y = np.cross(z, x)
    x = x / la.norm(x)
    y = y / la.norm(y)
    return np.matrix([[x[0], x[1], x[2], 0.0], 
                      [y[0], y[1], y[2], 0.0],
                      [z[0], z[1], z[2], 0.0],
                      [0.0, 0.0, 0.0, 1.0]])
    # return np.matrix([[x[0], y[0], z[0], 0.0], 
    #                   [x[1], y[1], z[1], 0.0],
    #                   [x[2], y[2], z[2], 0.0],
    #                   [0.0, 0.0, 0.0, 1.0]])

arcball_test = arcball_matrix(forward_test, up_test)
print "arcball "
print arcball_test

def homogeneous_times(matrix, vec3):
    homogeneous_point = np.matrix([vec3[0], vec3[1], vec3[2], 1.0])
    vec4 = matrix * np.transpose(homogeneous_point)
    homogeneous_return = np.array([vec4[0]/vec4[3], vec4[1]/vec4[3], vec4[2]/vec4[3]])
    homogeneous_return = np.reshape(homogeneous_return, (1, 3))
    return np.array([homogeneous_return[0][0], homogeneous_return[0][1], homogeneous_return[0][2]])

# PA world to screen
print (PA_test - look_test)
eye_space_point_test = np.array([0.0, 0.0, -1.0]) + homogeneous_times(arcball_test, PA_test - look_test)
print "eye_space_point ", eye_space_point_test

image_space_point_test = homogeneous_times(proj_test, eye_space_point_test)
print "image_space_point ", image_space_point_test

look =  [0 0 0]
C =  [1 0 0]
forward =  [-1  0  0]
up =  [0 1 0]
right =  [0 0 1]
arcball 
[[ 0.  0. -1.  0.]
 [-0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]]
[-1  0  0]
eye_space_point  [ 0.  0. -2.]
image_space_point  [ 0.         0.         0.9019019]


In [86]:
# Conversion from world space to camera screen space (from Unity: z,x,-y)

# Apex shot
# camera specs (position, forward, up, fov)
C = np.array([-92.9,-39.6,-0.8])
forward = np.array([-1.0,0.0,0.0])
up = np.array([0.0,0.0,-1.0])
look = C + forward
aspect = 16.0/9.0
fovY = 0.8726646
fovX = fovX_from_fovY(fovY, aspect)

print "look = ", look
print "C = ", C
print "forward = ", forward
print "up = ", up
right = np.cross(up, forward)
right / la.norm(right)
print "right = ", right

# scene specs
PA = np.array([-98.2,-40.6,-1.6])     # person A position
PB = np.array([-98.2,-38.6,-1.6])     # person B position

# PA world to screen
proj = perspective_matrix(fovY, aspect, 0.1, 100.0)
arcball = arcball_matrix(forward, up)
print arcball

print PA-look
eye_space_point = np.array([0.0, 0.0, -1.0]) + homogeneous_times(arcball, PA-look)
print "eye_space_point ", eye_space_point

image_space_point = homogeneous_times(proj, eye_space_point)
print "image_space_point ", image_space_point

look =  [-93.9 -39.6  -0.8]
C =  [-92.9 -39.6  -0.8]
forward =  [-1.  0.  0.]
up =  [ 0.  0. -1.]
right =  [ 0.  1.  0.]
[[-0. -1. -0.  0.]
 [ 0.  0. -1.  0.]
 [ 1. -0. -0.  0.]
 [ 0.  0.  0.  1.]]
[-4.3 -1.  -0.8]
eye_space_point  [ 1.   0.8 -5.3]
image_space_point  [ 0.22760098  0.32369917  0.96422838]


In [80]:
eye_space_point = np.array([0.0, 0.0, -1.0]) + homogeneous_times(arcball, PB - look)
print "eye_space_point ", eye_space_point

image_space_point = homogeneous_times(proj, eye_space_point)
print "image_space_point ", image_space_point

eye_space_point  [-1.   0.8 -5.3]
image_space_point  [-0.22760098  0.32369917  0.96422838]
